# Plan of Attack

1. Create sample (Done)
2. Create train/validation/test data set (Done)
3. Create simple vgg model (Done)
4. Get baseline performance by playing with learning rate (Done)
5. Work out data augmentation using simple model (Done)
6. Swap out baseline model with vgg and precalulate convolutional layers (Done)
7. Further fine tune model
8. submit to kaggle

## Create Sample and Train Validation Test Data Set

In [1]:
import os,sys,random, shutil
import numpy as np
#vgg is held in a subdirectory
sys.path.append('kernel')
import vgg16; reload(vgg16)
from vgg16 import Vgg16
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D,Flatten,Dropout
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam
import bcolz
import utils

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda0: GeForce GTX 970 (0000:01:00.0)


In [2]:
path = "../data/fish/"
os.listdir(path)

['precomp',
 'sample',
 'sample_submission_stg1.csv.zip',
 'sample_submission_stg2.csv.zip',
 'test',
 'test_stg1.zip',
 'test_stg2.7z',
 'train',
 'train (2).zip',
 'valid']

In [6]:
os.listdir(path + 'train/')

['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [22]:
#for each directory mk a directory in sample/train, sample/valid, valid
for d in os.listdir(path + 'train/'):
    os.mkdir(path + 'valid/' + d)

In [24]:
for d in os.listdir(path + 'train/'):
    imgs = os.listdir(path + 'train/' + d)
    shuf = np.random.permutation(imgs)
    print("For class " + d + " there were " + str(len(shuf)) + " images!" + " Moving " + str(len(shuf)//10) + "images\n")
#    for i in shuf[0:100]:
#        shutil.copyfile(path + 'train/' + d + '/' + i, path + 'sample/train/' + d + '/' + i)
#    for i in shuf[100:200]:
#        shutil.copyfile(path + 'train/' + d + '/' + i, path + 'sample/valid/' + d + '/' + i)
    for i in shuf[0:len(shuf) // 10]:
        shutil.move(path + 'train/' + d + '/' + i, path + 'valid/' + d + '/' + i)

For class ALB there were 1719 images! Moving 171images

For class BET there were 200 images! Moving 20images

For class DOL there were 117 images! Moving 11images

For class LAG there were 67 images! Moving 6images

For class NoF there were 465 images! Moving 46images

For class OTHER there were 299 images! Moving 29images

For class SHARK there were 176 images! Moving 17images

For class YFT there were 734 images! Moving 73images



## Create Simple Model

In [14]:
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D,Flatten,Dropout
from keras.preprocessing import image
from keras.optimizers import SGD, RMSprop, Adam

In [22]:
#create model with 2 convolutional layers
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(8, activation='softmax')
        ])

## Baseline Performance and Image Augmentation Stage 1

In [23]:
model.compile(optimizer=Adam(lr=.001), 
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# generate image data generators for sample
gen1 = image.ImageDataGenerator(shear_range=.17,rotation_range=45, width_shift_range=.15, height_shift_range=.15)
gen2 = image.ImageDataGenerator()
s_test = gen1.flow_from_directory(directory=path + 'train',
                                 target_size=(224,224),
                                 class_mode = 'categorical')
s_valid = gen2.flow_from_directory(directory=path + 'validate',
                                 target_size=(224,224),
                                  class_mode='categorical')

Found 3458 images belonging to 8 classes.
Found 319 images belonging to 8 classes.


In [27]:
model.fit_generator(s_test,samples_per_epoch=s_test.nb_sample,
                    nb_epoch=5, validation_data=s_valid, 
                    nb_val_samples = s_valid.nb_sample)

Epoch 1/5
3458/3458 [==============================] - 139s - loss: 1.6542 - acc: 0.4783 - val_loss: 3.2031 - val_acc: 0.1881

In [28]:
model.fit_generator(s_test,samples_per_epoch=s_test.nb_sample,
                    nb_epoch=5, validation_data=s_valid, 
                    nb_val_samples = s_valid.nb_sample)

Epoch 1/5
3458/3458 [==============================] - 83s - loss: 0.9383 - acc: 0.6654 - val_loss: 0.7554 - val_acc: 0.7492

## Bring in VGG using what we learned

In [11]:
#vgg is held in a subdirectory
sys.path.append('kernel')
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [42]:
m = Vgg16()
gen1 = image.ImageDataGenerator(shear_range=.17,rotation_range=45, width_shift_range=.15, height_shift_range=.15)
gen2 = image.ImageDataGenerator()
s_test = gen1.flow_from_directory(directory=path + 'train',
                                 target_size=(224,224),
                                 class_mode = 'categorical',batch_size=16)
s_valid = gen2.flow_from_directory(directory=path + 'valid',
                                 target_size=(224,224),
                                  class_mode='categorical',batch_size=16)


Found 3458 images belonging to 8 classes.
Found 319 images belonging to 8 classes.


In [ ]:
m.finetune(s_test)
m.fit(s_test,s_valid,nb_epoch=10)

In [ ]:
m.fit(s_test,s_valid,nb_epoch=5)

## Precomputer convolutional layers

In [8]:
import bcolz
import utils

In [2]:
m = Vgg16()

In [3]:
m.model

In [4]:
#create model which contains only vggs convolutional layers
last_conv_index = [i for i,l in enumerate(m.model.layers) if type(l) is Convolution2D][-1]
conv_model = Sequential(m.model.layers[:last_conv_index])
#conv_model.summary()

In [3]:
#get generators that aren't shuffled for our various imgs and then generate using predict the data
train_gen = image.ImageDataGenerator(shear_range=.17,rotation_range=45, width_shift_range=.15, height_shift_range=.15)
val_gen =  image.ImageDataGenerator()
test_gen = image.ImageDataGenerator()

train_batches = train_gen.flow_from_directory(directory=path + 'train',
                                 target_size=(224,224),
                                 class_mode = 'categorical',batch_size=16, shuffle=False)

val_batches = val_gen.flow_from_directory(directory=path + 'valid',
                                 target_size=(224,224),
                                 class_mode = 'categorical',batch_size=16, shuffle=False)

test_batches = test_gen.flow_from_directory(directory=path + 'test',
                                 target_size=(224,224),
                                 class_mode = 'categorical',batch_size=16, shuffle=False)

#taken from fast.ai utils.py
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = utils.get_classes(path)

Found 3404 images belonging to 8 classes.
Found 373 images belonging to 8 classes.
Found 12153 images belonging to 1 classes.
Found 3404 images belonging to 8 classes.
Found 373 images belonging to 8 classes.
Found 12153 images belonging to 1 classes.


In [28]:
conv_train_feat = conv_model.predict_generator(train_batches, train_batches.nb_sample * 5)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

utils.save_array(path+'precomp/conv_train_feat.dat', conv_train_feat)
utils.save_array(path+'precomp/conv_val_feat.dat', conv_val_feat)
utils.save_array(path+'precomp/conv_test_feat.dat', conv_test_feat)

NameError: name 'conv_model' is not defined

In [9]:
train_labels = np.concatenate((trn_labels,trn_labels,trn_labels,trn_labels,trn_labels))
conv_test_feat = utils.get_array_generator(path+'precomp/conv_train_feat.dat',batch_size=16,labels=train_labels)
conv_val_feat = utils.get_array_generator(path+'precomp/conv_val_feat.dat', batch_size=16,labels=val_labels)
#conv_test_feat = utils.load_array(path+'precomp/conv_train_feat.dat')
#conv_val_feat = utils.load_array(path+'precomp/conv_val_feat.dat')
train_size = bcolz.open(path +'precomp/conv_train_feat.dat').shape[0]
val_size = bcolz.open(path +'precomp/conv_val_feat.dat').shape[0]

In [5]:
dense_model = Sequential()
#add our vgg style dense layers
dense_model.add(MaxPooling2D((2, 2), strides=(2, 2),input_shape=(512,16,16)))
dense_model.add(Flatten())
dense_model.add(Dense(4096, activation='relu'))
dense_model.add(Dropout(0.2))
dense_model.add(Dense(4096, activation='relu'))
dense_model.add(Dropout(0.2))
dense_model.add(Dense(8, activation='softmax'))

In [12]:
dense_model.compile(optimizer=Adam(lr=.00001), loss='categorical_crossentropy', 
                    metrics=['accuracy'])


In [13]:
#I need to write a function that takes val_classes and conv_val_feat and generates a tuple of batches
# of shape ((16,512,16,16),(16,8))
dense_model.fit_generator(conv_test_feat, samples_per_epoch=train_size, 
                          nb_epoch=1, validation_data=conv_val_feat, nb_val_samples=val_size)

Epoch 1/1


GpuArrayException: out of memory
Apply node that caused the error: GpuDot22(InplaceGpuDimShuffle{1,0}.0, GpuElemwise{Composite{((i0 * Composite{((i0 * i1 * i2) + i3)}(i1, i2, i3, i4)) + (i5 * Composite{((i0 * i1 * i2) + i3)}(i1, i2, i3, i4) * sgn(i6)))}}[(0, 2)]<gpuarray>.0)
Toposort index: 133
Inputs types: [GpuArrayType<None>(float32, (False, False)), GpuArrayType<None>(float32, (False, False))]
Inputs shapes: [(32768, 16), (16, 4096)]
Inputs strides: [(4, 131072), (16384, 4)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[GpuElemwise{Composite{((i0 * i1) + (i2 * sqr(i3)))}}[(0, 1)]<gpuarray>(InplaceGpuDimShuffle{x,x}.0, <GpuArrayType<None>(float32, (False, False))>, InplaceGpuDimShuffle{x,x}.0, GpuDot22.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimize

In [45]:
conv_test_feat.nb_sample

AttributeError: 'generator' object has no attribute 'nb_sample'

In [20]:
i = 0
for a,b in conv_test_feat:
    if (len(a) != len(b)):
        print("on iteration " + str(i) + " we failed to match\n")
        break
    i = i + 1
#len(next(conv_test_feat)[1])

on iteration 212 we failed to match



In [18]:
len(next(conv_val_feat))

2

In [8]:
reload(utils)

<module 'utils' from 'utils.py'>

In [27]:
len(train_labels)

17020